In [1]:
import os
import sys
root_dir = os.path.abspath(os.path.join(os.path.dirname('../full_portfolio_analysis.ipynb'), '..'))
os.chdir(root_dir)

sys.path.insert(0, os.path.join(root_dir, 'src'))

In [4]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover

import pandas as pd
import pandas_ta as pandas_ta
import MetaTrader5 as mt5
import pandas as pd
from backtest.get_data import get_data
import pytz
from datetime import datetime
import talib as ta
import random
random.seed(42)


In [ ]:
# establish connection to the MetaTrader 5 terminal
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()
 
# get all symbols
symbols = mt5.symbols_get()


groups = []
for symbol in symbols:
    group = symbol.path.split('\\')[0]
    if not group in groups:
        groups.append(group)
groups


In [ ]:
tickers = [symbol.path.split('\\')[1] for symbol in symbols if 'Stocks_group' in symbol.path and symbol.spread < 15]
tickers

In [ ]:

tickers = [
    'AUS200m',
    'DE30m',
    'FR40m',
    'HK50m',
    'JP225m',
    'STOXX50m',
    'UK100m',
    'US30m',
    'US500m',
    'USTECm',
]


# tickers = [symbol.path.split('\\')[1] for symbol in symbols if 'Indices_group' in symbol.path and symbol.spread < 15]
print(tickers)


# Establecer la zona horaria a UTC
timezone = pytz.timezone("Etc/UTC")

# Crear objetos 'datetime' en zona horaria UTC
date_from = datetime(2014, 10, 1, tzinfo=timezone)
date_to = datetime(2024, 9, 1, tzinfo=timezone)

intervals = [
    # mt5.TIMEFRAME_M1,
    # mt5.TIMEFRAME_M2,
    # mt5.TIMEFRAME_D1,
    # mt5.TIMEFRAME_M3,
    # mt5.TIMEFRAME_M4,
    # mt5.TIMEFRAME_M5,
    mt5.TIMEFRAME_H1,
    mt5.TIMEFRAME_H2,
    mt5.TIMEFRAME_H4,
    # mt5.TIMEFRAME_M15,
    # mt5.TIMEFRAME_M30,
    # mt5.TIMEFRAME_M30,
]

symbols = get_data(tickers, intervals, date_from, date_to)

In [ ]:
max_start_date = None
intervals_start_dates = {}

for interval in intervals:
    for ticker in tickers:
        if not max_start_date or symbols[ticker][interval].index.min() > max_start_date:
            max_start_date = symbols[ticker][interval].index.min()
        
    intervals_start_dates[interval] = max_start_date

intervals_start_dates



In [ ]:
from pandas import Timestamp
limit_date_train = Timestamp('2022-10-01 00:00:00')
limit_date_train

In [14]:
class Bbands(Strategy):
    risk=3
    bbands_timeperiod = 50
    bband_std = 1.5
    sma_period = 200
    b_open_threshold = 0.85
    b_close_threshold = 0.5

    def init(self):
        
        self.sma = self.I(
            ta.SMA, self.data.Close, timeperiod=self.sma_period
        )

        self.upper_band, self.middle_band, self.lower_band = self.I(
            ta.BBANDS, self.data.Close, 
            timeperiod=self.bbands_timeperiod, 
            nbdevup=self.bband_std, 
            nbdevdn=self.bband_std
        )

    def next(self):
        actual_close = self.data.Close[-1]
        b_percent = (actual_close - self.lower_band[-1]) / (self.upper_band[-1] - self.lower_band[-1])
        
        if self.position:
            if self.position.is_long:
                if b_percent >= self.b_close_threshold:
                    self.position.close()

            if self.position.is_short:
                if b_percent <= 1 - self.b_close_threshold:
                    self.position.close()

        else:

            if b_percent <= 1 - self.b_open_threshold and actual_close > self.sma:
                self.buy(size=self.risk / 100)
                
            if b_percent >= self.b_open_threshold and actual_close < self.sma:
                self.sell(size=self.risk / 100)
            

In [58]:
from backbone.utils.general_purpose import calculate_units_size, diff_pips

class BPercent(Strategy):
    risk= 1
    bbands_timeperiod = 50
    bband_std = 1.5
    sma_period = 200
    b_open_threshold = 0.95
    b_close_threshold = 0.5
    atr_multiplier = 1.5
    pip_value = 0.1

    def init(self):
        
        self.sma = self.I(
            ta.SMA, self.data.Close, timeperiod=self.sma_period
        )

        self.upper_band, self.middle_band, self.lower_band = self.I(
            ta.BBANDS, self.data.Close, 
            timeperiod=self.bbands_timeperiod, 
            nbdevup=self.bband_std, 
            nbdevdn=self.bband_std
        )
        
        self.atr = self.I(ta.ATR, self.data.High, self.data.Low, self.data.Close)

    def next(self):
        actual_close = self.data.Close[-1]
        b_percent = (actual_close - self.lower_band[-1]) / (self.upper_band[-1] - self.lower_band[-1])
        
        if self.position:
            if self.position.is_long:
                if b_percent >= self.b_close_threshold:
                    self.position.close()

            if self.position.is_short:
                if b_percent <= 1 - self.b_close_threshold:
                    self.position.close()

        else:

            if b_percent <= 1 - self.b_open_threshold and actual_close > self.sma[-1]:
                sl_price = self.data.Close[-1] - self.atr_multiplier * self.atr[-1]
                
                pip_distance = diff_pips(
                    self.data.Close[-1], 
                    sl_price, 
                    pip_value=self.pip_value
                )
                
                units = calculate_units_size(
                    account_size=self.equity, 
                    risk_percentage=self.risk, 
                    stop_loss_pips=pip_distance, 
                    pip_value=self.pip_value
                )
                
                self.buy(
                    size=units,
                    sl=sl_price
                )
                
            if b_percent >= self.b_open_threshold and actual_close < self.sma[-1]:
                sl_price = self.data.Close[-1] + self.atr_multiplier * self.atr[-1]
                
                pip_distance = diff_pips(
                    self.data.Close[-1], 
                    sl_price, 
                    pip_value=self.pip_value
                )
                
                units = calculate_units_size(
                    account_size=self.equity, 
                    risk_percentage=self.risk, 
                    stop_loss_pips=pip_distance, 
                    pip_value=self.pip_value
                )
                
                self.sell(
                    size=units,
                    sl=sl_price
                )

In [19]:
import itertools
import numpy as np
from sklearn.linear_model import LinearRegression

strategies = [
    Bbands, BPercent
]

experiments = parameter_combinations = list(itertools.product(
    tickers, intervals, strategies
))

performance = pd.DataFrame()

for ticker, interval, strategy in experiments:
    start_date = intervals_start_dates[interval]

    frac_df = symbols[ticker][interval].loc[start_date:limit_date_train] * 0.01

    bt_train = Backtest(
        frac_df, 
        strategy,
        commission=7e-4,
        cash=100_000, 
        margin=1/30
    )
    
    stats = bt_train.run()
    
    equity_curve = stats._equity_curve['Equity'].values    
    x = np.arange(len(equity_curve)).reshape(-1, 1)
    reg = LinearRegression().fit(x, equity_curve)
    stability_ratio = reg.score(x, equity_curve)

    df_stats = pd.DataFrame({
        'strategy':[strategy.__name__],
        'ticker':[ticker],
        'interval':[interval],
        'stability_ratio':[stability_ratio],
        'return':[stats['Return [%]']],
        'final_eq':[stats['Equity Final [$]']],
        'drawdown':[stats['Max. Drawdown [%]']],
        'drawdown_duration':[stats['Max. Drawdown Duration']],
        'win_rate':[stats['Win Rate [%]']], 
        'sharpe_ratio':[stats['Sharpe Ratio']],
        'trades':[stats['# Trades']],
        'avg_trade_percent':[stats['Avg. Trade [%]']],
        'exposure':[stats['Exposure Time [%]']],
        'final_equity':[stats['Equity Final [$]']],
        'Duration':[stats['Duration']],

    })

    performance = pd.concat([performance, df_stats])

performance['return/dd'] = performance['return'] / -performance['drawdown']
performance['drawdown'] = -performance['drawdown']
performance['custom_metric'] = (performance['return'] / (1 + performance.drawdown)) * np.log(1 + performance.trades)

performance.drawdown_duration = pd.to_timedelta(performance.drawdown_duration)

performance.drawdown_duration = performance.drawdown_duration.dt.days

In [ ]:

filter_performance = performance[
    (performance['return']>0) 
    # & (performance['return/dd']>1) 
    & (performance['strategy'] == 'BPercent')
    # & (performance['interval'] <= 16385)
].sort_values(by=['return/dd'], ascending=[False]).drop_duplicates(subset=['ticker'], keep='first')

portfolio = filter_performance.ticker.tolist()
intervals = filter_performance.interval.values.tolist()

display(filter_performance)

# portfolio

In [ ]:
for ticker, interval in zip(portfolio, intervals):
    frac_df = symbols[ticker][interval].loc[start_date:limit_date_train] * 0.01
    
    bt_train = Backtest(
        frac_df, 
        BPercent,
        commission=7e-4,
        cash=100_000, 
        margin=1/30
    )

    stats = bt_train.run()
    
    print(stats)

    bt_train.plot(filename=f'./{ticker}.html', resample=False)

# WFO

In [ ]:
from backtest.utils import plot_full_equity_curve, walk_forward

def optim_func_2(stats):
    equity_curve = stats._equity_curve['Equity'].values    
    x = np.arange(len(equity_curve)).reshape(-1, 1)
    reg = LinearRegression().fit(x, equity_curve)
    stability_ratio = reg.score(x, equity_curve)
    
    return (stats['Return [%]'] /  (1 + (-1*stats['Max. Drawdown [%]']))) * np.log(1 + stats['# Trades']) * stability_ratio
    
stats_per_symbol = {}
lookback_bars = 1000
validation_bars = 250
warmup_bars = 200

wfo_performance = pd.DataFrame()
for ticker, interval in zip(['US30m', 'USTECm'], [16388, 16388]):
    
    params = {
        'b_open_threshold' : [0.9, 0.95, 0.1, 1.5],
        'b_close_threshold': [0.5, 0.6, 0.8, 0.9],
        'atr_multiplier': list(np.arange(1.5, 4, 0.2).round(2)),
        'maximize': optim_func_2
    }
    
    fracc_df = symbols[ticker][interval] * 0.01

    wfo_stats = walk_forward(
        BPercent,
        fracc_df, 
        lookback_bars=lookback_bars,
        validation_bars=validation_bars,
        warmup_bars=warmup_bars, 
        params=params,
        commission=7e-4, 
        margin=1/30, 
        cash=10_000,
        verbose=False
    )
    
    stats_per_symbol[ticker] = {}
    stats_per_symbol[ticker][interval] = wfo_stats
    
    df_equity = wfo_stats['_equity']
    
    plot_full_equity_curve(df_equity, title=f'{ticker}, {interval}')
    
    # Calculo el stability ratio
    x = np.arange(df_equity.shape[0]).reshape(-1, 1)
    reg = LinearRegression().fit(x, df_equity.Equity)
    stability_ratio = reg.score(x, df_equity.Equity)

    # Extraigo metricas
    df_stats = pd.DataFrame({
        'strategy':[strategy.__name__],
        'ticker':[ticker],
        'interval':[interval],
        'stability_ratio':[stability_ratio],
        'return':[wfo_stats['Return [%]']],
        'drawdown':[wfo_stats['Max. Drawdown [%]']],
        'final_eq':[wfo_stats['Equity Final [$]']],
        'drawdown_duration':[wfo_stats['Max. Drawdown Duration']],
        'win_rate':[wfo_stats['Win Rate [%]']], 
        'sharpe_ratio':[wfo_stats['Sharpe Ratio']],
        'trades':[wfo_stats['# Trades']],
        'avg_trade_percent':[wfo_stats['Avg. Trade [%]']],
        'exposure':[wfo_stats['Exposure Time [%]']],
        'final_equity':[wfo_stats['Equity Final [$]']],
        'Duration':[wfo_stats['Duration']],

    })

    wfo_performance = pd.concat([wfo_performance, df_stats])

wfo_performance['return/dd'] = wfo_performance['return'] / -wfo_performance['drawdown']
wfo_performance['drawdown'] = -wfo_performance['drawdown']
wfo_performance['custom_metric'] = (wfo_performance['return'] / (1 + wfo_performance.drawdown)) * np.log(1 + wfo_performance.trades)

wfo_performance.drawdown_duration = pd.to_timedelta(wfo_performance.drawdown_duration)
wfo_performance.drawdown_duration = wfo_performance.drawdown_duration.dt.days

wfo_performance.sort_values(by='return/dd', ascending=False)